In [4]:
import numpy as np
from sympy.matrices import Matrix
from collections import Counter
import bisect

In [5]:
class SiteBasis:
    def __init__(self, N):
        self._N = N
        self._ops = self.gen_basis()
        self._ops.sort()
        self._opset = set(self._ops)
   
    def full_rank(self, word):
        r = 0
        for i, op in enumerate(reversed(word)):
            if op not in 'IXYZ':
                raise ValueError(f'{word} must have only IXYZ')
            r += (4 ** i) * ('IXYZ'.find(op))
        return r

    def full_unrank(self, pos):
        word = [None] * self._N
        for i in range(self._N):
            word[i] = 'IXYZ'[pos % 4]
            pos //= 4
        return ''.join(word[::-1])

    def enforce_rotation(self):
        exclude = set()
        L = self._N
        for i in range(4 ** L):
            if i in exclude:
                continue
            op = self.full_unrank(i)
            op2 = op
            for j in range(L - 1):
                op = op[1:] + op[0]
                if op == op2:
                    break
                exclude.add(self.full_rank(op))
        small_basis = []
        for i in range(4 ** L):
            if i in exclude:
                continue
            small_basis.append(self.full_unrank(i))
        return small_basis

    def enforce_reflection(self, basis):
        exclude = set()
        bset = set(basis)
        for word in bset:
            if word in exclude:
                continue
            rword = word[::-1]
            if word == rword:
                continue
            if rword not in bset:
                rword2 = rword[1:] + rword[0]
                while rword2 not in bset:
                    if rword2 == rword:
                        break
                    rword2 = rword2[1:] + rword2[0]
                if word != rword2:
                    exclude.add(rword2)
            else:
                exclude.add(rword)
        smaller_basis = []
        for word in basis:
            if word in exclude:
                continue
            smaller_basis.append(word)
        return smaller_basis

    def gen_basis(self):
        small_basis = []
        L = self._N
        for i in range(4 ** L):
            small_basis.append(self.full_unrank(i))
        return small_basis
        # small_basis = self.enforce_rotation()
        # return self.enforce_reflection(small_basis)
    
    def size(self):
        return len(self._ops)
    
    def rank(self, word: str):
        if len(word) != self._N:
            raise ValueError(f'{word} must be of length {self._N}')
        word = self.normalize(word)
        return self._ops.index(word)
        # return bisect.bisect_left(self._ops, word)

    def unrank(self, pos: int):
        # word = self.full_unrank(pos)
        # return self._ops[self.rank(word)]
        return self._ops[pos]

    def normalize(self, word):
        if len(word) != self._N:
            raise ValueError(f'{word} must be of length {self._N}')
        if word not in self._opset:
            word2 = word[1:] + word[0]
            while word2 not in self._opset:
                if word2 == word:
                    return self.normalize(word[::-1])
                word2 = word2[1:] + word2[0]
            return word2
        return word

In [6]:
def basis_commutator(o1, o2):
    commutation_table = [
        [(0, ''),(0, ''),(0, ''),(0, '')],
        [(0, ''),(0, ''),(2, 'Z'),(-2, 'Y')],
        [(0, ''),(-2, 'Z'),(0, ''),(2, 'X')],
        [(0, ''),(2, 'Y'),(-2, 'X'),(0, '')],
    ] # everything here must be multiplied with 1j
    return commutation_table['IXYZ'.find(o1)]['IXYZ'.find(o2)]

def basis_anticommutator(o1, o2):
    anticommutation_table = [
        [(2, 'I'),(2, 'X'),(2, 'Y'),(2, 'Z')],
        [(2, 'X'),(2, 'I'),(0, ''),(0, '')],
        [(2, 'Y'),(0, ''),(2, 'I'),(0, '')],
        [(2, 'Z'),(0, ''),(0, ''),(2, 'I')],
    ]
    return anticommutation_table['IXYZ'.find(o1)]['IXYZ'.find(o2)]

def basis_product(o1, o2):
    product_table = [
        [(1, 'I'),(1, 'X'),(1, 'Y'),(1, 'Z')],
        [(1, 'X'),(1, 'I'),(1j, 'Z'),(-1j, 'Y')],
        [(1, 'Y'),(-1j, 'Z'),(1, 'I'),(1j, 'X')],
        [(1, 'Z'),(1j, 'Y'),(-1j, 'X'),(1, 'I')],
    ]
    return product_table['IXYZ'.find(o1)]['IXYZ'.find(o2)]

def anticommutator(word1, word2):
    if len(word1) != len(word2):
        raise ValueError(f'{word1} and {word2} do not have same length')
    if len(word1) == 1:
        c, o = basis_anticommutator(word1, word2)
        return Counter({o: c})
    expression = Counter()
    e_1 = commutator(word1[0], word2[0])
    e_2 = commutator(word1[1:], word2[1:])
    e_3 = anticommutator(word1[0], word2[0])
    e_4 = anticommutator(word1[1:], word2[1:])
    for o1, c1 in e_1.items():
        for o2, c2 in e_2.items():
            expression[o1 + o2] -= 0.5 * c1 * c2 # since c1 * 1j * c2 * 1j = - c1 * c2
    for o1, c1 in e_3.items():
        for o2, c2 in e_4.items():
            expression[o1 + o2] += 0.5 * c1 * c2
    return expression

def commutator(word1: str, word2: str):
    if len(word1) != len(word2):
        raise ValueError(f'{word1} and {word2} do not have same length')
    if len(word1) == 1:
        c, o = basis_commutator(word1, word2)
        return Counter({o: c})
    expression = Counter()
    e_1 = commutator(word1[0], word2[0])
    e_2 = anticommutator(word1[1:], word2[1:])
    e_3 = anticommutator(word1[0], word2[0])
    e_4 = commutator(word1[1:], word2[1:])
    for o1, c1 in e_1.items():
        for o2, c2 in e_2.items():
            expression[o1 + o2] += 0.5 * c1 * c2
    for o1, c1 in e_3.items():
        for o2, c2 in e_4.items():
            expression[o1 + o2] += 0.5 * c1 * c2
    # whatever is returned, needs to be multiplied by 1j
    return expression

def product(word1, word2):
    if len(word1) != len(word2):
        raise ValueError(f'{word1} and {word2} do not have same length')
    word = []
    coeff = 1
    for o1, o2 in zip(word1, word2):
        c, op = basis_product(o1, o2)
        word.append(op)
        coeff *= c
    # coeff can be real or complex
    return (coeff, ''.join(word))

In [9]:
class BasisReduction():
    def __init__(self, A, b):
        self._A = A.copy()
        self._b = b.copy()
        self._Arref, self._brref = Matrix(A).rref_rhs(Matrix(b))
        self._pivots = Matrix(A).rref()[1]
        self._free = sorted(list(set(range(A.shape[1])) - set(self._pivots))) # This is with the original numbering
        self._sz = len(self._free)
        self._reductions = [[Counter(), 0.0] for _ in range(A.shape[1])]
        self._init_free() # Initialize mappings for free variables
        self._generate_reductions() # Initialize mappings for pivot variables

    def _init_free(self):
        for i, p in enumerate(self._free):
            self._reductions[p][0][i] += 1.0
            self._reductions[p][1] = 0.0
    
    def _generate_reductions(self):
        crow = len(self._pivots) - 1
        for pcol in reversed(self._pivots):
            # Create mapping
            self._reductions[pcol][1] += np.double(self._brref[crow])
            for ccol in range(pcol + 1, self._A.shape[1]):
                if not np.isclose(self._Arref[crow, ccol], 0.0):
                    coeff = np.double(-self._Arref[crow, ccol]) # minus sign to transfer to other side of equality
                    self._reductions[pcol][1] += coeff * self._reductions[ccol][1]
                    for freew, c in self._reductions[ccol][0].items():
                        self._reductions[pcol][0][freew] += coeff * c
            # Update currow
            crow -= 1

    def size(self):
        return self._sz

    def reduce(self, p):
        # Return a mapping of the p-th element of the original basis
        # to a linear combination + constant offset of the new basis
        # Remember to take care of the numbering
        return self._reductions[p]

In [20]:
def schwinger_dyson(basis, n, hamil: list, word1: str):
    C = np.zeros((1, n))
    expression = Counter()
    for coeff, w in hamil:
        terms = commutator(w, word1)
        for op, c in terms.items():
            if len(op) != len(word1):
                continue
            expression[op] += coeff * c
    for op, c in expression.items():
        C[0, basis.rank(op)] += c
    return C

def construct_hamil(L, h):
    hamil = []
    for i in range(L):
        t1 = None
        if i<L-1:
            t1 = 'I'*i + 'XX' + 'I'*(L-i-2)
        else:
            t1 = 'X' + 'I'*(L-2) + 'X'
        t2 = 'I'*i+'Z'+'I'*(L-i-1)
        hamil.extend([(-1, t1), (-h, t2)])
    if L == 2:
        hamil = [(-h, 'IZ'), (-h, 'ZI'), (-1, 'XX')]
    return hamil

def construct_lin(L, basis, hamil):
    n = basis.size()
    cons = [np.zeros((1, n))]
    cons[0][0, basis.rank('I' * L)] = 1.0
    for i in range(1, n):
        cons.append(schwinger_dyson(basis, n, hamil, basis.unrank(i)))
    A = np.vstack(cons)
    b = np.zeros((A.shape[0], 1))
    b[0, 0] = 1.0 # Normalization
    return A, b

# Remember that all expectation values are real in the Ising model
def create_problem(L):
    basis = SiteBasis(L)
    n = basis.size()
    H = construct_hamil(L, 1)
    A, b = construct_lin(L, basis, H)
    redbas = BasisReduction(A, b)
    c = np.zeros((redbas.size(), 1))
    offset = 0
    for coeff, word in H:
        # rew will be a dict with mapping {rank in reduced basis: coefficient}
        p2 = basis.rank(word)
        rew, const = redbas.reduce(p2)
        offset += coeff * const
        for p, c2 in rew.items():
            c[p, 0] += coeff * c2
    return A, b, c, offset, redbas, basis

In [23]:
L = 3
A, b, c, offset, redbas, basis = create_problem(L)

In [24]:
for i in redbas._free:
    print(basis.unrank(i))

IZZ
YXI
YYZ
YZY
ZII
ZIX
ZIZ
ZXI
ZXX
ZYX
ZYY
ZYZ
ZZI
ZZY
ZZZ
